## Introduction

This notebook prepares the data from EdGap and NCES for analysis.

### Importing libraries

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')

from sklearn.model_selection import train_test_split
from sklearn.impute import IterativeImputer

from google.colab import files

### Sources of the Data

This project utilizes two data sets. The primary data set is the EdGap data set from [EdGap.org](https://www.edgap.org/#5/37.875/-96.987). This data set from 2016 includes information about average ACT or SAT scores for schools and several socioeconomic characteristics of the school district. The secondary data set is basic information about each school from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp).



#### EdGap

All socioeconomic data (household income, unemployment, adult educational attainment, and family structure) are from the Census Bureau's American Community Survey. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) report that ACT and SAT score data is from each state's department of education or some other public data release. The nature of the other public data release is not known.

The quality of the census data and the department of education data can be assumed to be reasonably high. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) do not indicate that they processed the data in any way. The data were assembled by the [EdGap.org](https://www.edgap.org/#5/37.875/-96.987) team, so there is always the possibility for human error. Given the public nature of the data, we would be able to consult the original data sources to check the quality of the data if we had any questions.

In [70]:
edgap = pd.read_excel('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/education/EdGap_data.xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


#### NCES

The school information data is from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp). This data set consists of basic identifying information about schools and can be assumed to be of reasonably high quality. As for the EdGap.org data, the school information data is public, so we would be able to consult the original data sources to check the quality of the data if we had any questions.

In [71]:
!wget https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0
school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding='unicode_escape')

--2023-04-25 02:50:56--  https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv [following]
--2023-04-25 02:50:57--  https://www.dropbox.com/s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc02e594ac8dc1c6e7b6e082ac0c.dl.dropboxusercontent.com/cd/0/inline/B61AbD4PzKorJlLkk5rJjTqxpix0cF6rNkvqXIJ_9S6xiTZ4JjJlphWCkP6xsSvW4wEAcTc9fUpEg8OI2xcWvIolv3aKD2nQkHcfJWQ2H1B8N9X5QY8lZAHeR_5N5u8cxB5kZiKz6CSUPtElYmL3l_mnqKTxdu0zHNLPKraYmtdPcA/file# [following]
--2023-04-25 02:50:57--  https://uc02e594ac8dc1c6e7b6e082ac0c.dl.dropboxusercontent.com/cd/0/inline/B61AbD

<ipython-input-71-7f0f862228ee>:2: DtypeWarning: Columns (6,9,14,15,18,19,21,22,25,26,29,31,35,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding='unicode_escape')


### Inspecting the contents
Here, we look at what each data set looks like.

In [72]:
edgap.info()
edgap.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7986 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   NCESSCH School ID                                7986 non-null   int64  
 1   CT Unemployment Rate                             7972 non-null   float64
 2   CT Pct Adults with College Degree                7973 non-null   float64
 3   CT Pct Childre In Married Couple Family          7961 non-null   float64
 4   CT Median Household Income                       7966 non-null   float64
 5   School ACT average (or equivalent if SAT score)  7986 non-null   float64
 6   School Pct Free and Reduced Lunch                7986 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 436.9 KB


,NCESSCH School ID,CT Unemployment Rate,CT Pct Adults with College Degree,CT Pct Childre In Married Couple Family,CT Median Household Income,School ACT average (or equivalent if SAT score),School Pct Free and Reduced Lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [73]:
school_info.info()
school_info.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102183 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102183 non-null  object 
 1   FIPST                102183 non-null  int64  
 2   STATENAME            102183 non-null  object 
 3   ST                   102183 non-null  object 
 4   SCH_NAME             102183 non-null  object 
 5   LEA_NAME             102183 non-null  object 
 6   STATE_AGENCY_NO      102183 non-null  object 
 7   UNION                2533 non-null    float64
 8   ST_LEAID             102183 non-null  object 
 9   LEAID                102183 non-null  object 
 10  ST_SCHID             102183 non-null  object 
 11  NCESSCH              102181 non-null  float64
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3         

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,...,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2016-2017,1,ALABAMA,AL,Sequoyah Sch - Chalkville Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
1,2016-2017,1,ALABAMA,AL,Camps,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
2,2016-2017,1,ALABAMA,AL,Det Ctr,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
3,2016-2017,1,ALABAMA,AL,Wallace Sch - Mt Meigs Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
4,2016-2017,1,ALABAMA,AL,McNeel Sch - Vacca Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported


### Converting data types
For the NCES data set, we ensure that the school ID is an int.

In [74]:
school_info['NCESSCH'] = school_info['NCESSCH'].fillna(0).astype('int')

### Removing unnecessary parts of the data
Only the columns that are useful to us are kept.

In [75]:
school_info = school_info[['SCHOOL_YEAR', 'NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL']]

## Renaming the columns
The columns are renamed to use snake case.

In [76]:
edgap = edgap.rename(columns={
    'NCESSCH School ID': 'id',
    'CT Pct Adults with College Degree': 'percent_college',
    'CT Unemployment Rate': 'rate_unemployment',
    'CT Pct Childre In Married Couple Family': 'percent_married',
    'CT Median Household Income': 'median_income',
    'School ACT average (or equivalent if SAT score)': 'average_act',
    'School Pct Free and Reduced Lunch': 'percent_lunch'
})
school_info = school_info.rename(columns={
    'SCHOOL_YEAR': 'year',
    'NCESSCH': 'id',
    'MSTATE': 'state',
    'MZIP': 'zip_code',
    'SCH_TYPE_TEXT': 'school_type',
    'LEVEL': 'school_level'
})

### Joining the data frames
The data frames `edgap` and `school_info` are joined, using their common values for `id`.

In [77]:
df = edgap.merge(school_info, on='id', how='left')

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,2016-2017,DE,19804,Regular School,High
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,2016-2017,DE,19709,Regular School,High
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,2016-2017,DE,19709,Regular School,High
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,2016-2017,DE,19958,Regular School,High
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,2016-2017,DE,19934,Regular School,High
...,...,...,...,...,...,...,...,...,...,...,...,...
7981,560530200294,0.034549,0.590780,0.720077,64891.0,18.500000,0.232156,2016-2017,WY,82902,Regular School,High
7982,560569000311,0.069538,0.633860,0.808841,44603.0,22.300000,0.083871,2016-2017,WY,82833,Regular School,High
7983,560569500360,0.035159,0.764566,0.776570,44896.0,21.500000,0.184829,2016-2017,WY,82801,Regular School,High
7984,560576200324,0.063877,0.670532,0.879324,92134.0,19.700000,0.112583,2016-2017,WY,82935,Regular School,High


### Handling out-of-range values

The minimum possible ACT score is 1, so any schools with average scores below that are filtered out.

In [79]:
df = df[df['average_act'] >= 1]

A percentage value should only be between 0 and 1 in a case like `percent_lunch`. Nonsense values of `percent_lunch` are set to `np.nan`.

In [78]:
df.loc[(df['percent_lunch'] < 0) | (df['percent_lunch'] > 1), 'percent_lunch'] = np.nan

### Narrowing the data further
Since we're investigating ACT scores, only high schools will be looked at. We'll also filter out any schools that aren't "regular", as the criteria for other types of schools is unclear.

In [80]:
df = df[df['school_level'] == 'High']
df = df[df['school_type'] == 'Regular School']

### Imputing missing values

The test set approach is used to compare models predicting `average_act`. 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/ML_dataset_training_validation_test_sets.png/800px-ML_dataset_training_validation_test_sets.png)

The data are split into training and testing sets, keeping 20% of the data for the test set.

In [81]:
x_columns = ['rate_unemployment', 'percent_college', 'percent_married', 'median_income', 'percent_lunch']
y_columns = ['average_act']
X_train, X_test, y_train, y_test = train_test_split(df[x_columns], df[y_columns], test_size=0.2)

An `IterativeImputer` is used to replace missing values in the columns corresponding to predictor variables in the analysis.

In [82]:
imputer = IterativeImputer()
imputer.fit(X_train.loc[:, x_columns])
X_train.loc[:, x_columns] = imputer.transform(X_train.loc[:,x_columns])
X_test.loc[:, x_columns] = imputer.transform(X_test.loc[:,x_columns])

df_train = X_train.join(y_train)

### Exporting the clean data
A `.csv` file is created from our new data frame.

In [83]:
df.to_csv('clean_school_info.csv', encoding = 'utf-8-sig', index=False) 
files.download('clean_school_info.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>